In [ ]:
  
# SKT Brain의 KoBERT, 학습 및 테스트 데이터셋만 따로 준비 = 카카오 브런치[text, label]
# SKT Brain github 주소는 다음과 같습니다. https://github.com/SKTBrain/KoBERT

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

     |████████████████████████████████| 46.9 MB 73 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595742 sha256=eefa5dedb62759128f714b193397a6c475cafc031168fedc26da5852cc644a68
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.1 MB/s 
     |████████████████████████████████| 769 kB 4.4 MB/s 
     |████████████████████████████████| 3.0 MB 36.0 MB/s 
     |████████████████████████████████| 895 kB 48.1 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-q_25ueqy
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# GPU 사용 시
device = torch.device("cuda:0")

# BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 데이터셋 불러오기
import pandas as pd
data1 = pd.read_excel('/content/drive/My Drive/nlp/0to499_labeling.xlsx', '합본')

In [ ]:
# data 확인
data1.head()

,문예진,이상민,정수연,정승연,황의린,평균,표준편차,상품명,리뷰,index
0,5.0,5,5.0,5,5,5.0,0.000000,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),이브랜드는 올리브영 입점전부터 써본브랜드입니다\n 수분크림구매시에 성분이 중요한데요...,0
1,5.0,5,5.0,5,5,5.0,0.000000,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),전에 어찌어찌하다 자작나무 수분크림 샘플을 받아 잘 쓰고 있는 중에\n 1+1행사한...,1
2,4.0,5,5.0,5,4,4.6,0.547723,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),올리브영 대표 클린뷰티! 라운드랩!\n \n # 라운드랩 자작나무 수분 크림 \n ...,2
3,3.0,4,4.0,4,4,3.8,0.447214,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),💚수분감\n 진짜 내가 찾던 수분만 있는 제품.. 유분은 기름종이 보시면 알겠지만 ...,3
4,3.0,5,4.0,4,4,4.0,0.707107,★에바 P!CK★라운드랩 자작나무 수분크림 1+1기획(+수분패드 10매),원쁠원 행사 넘 좋아요 이번달 끝나기 전에 하나 더 사놓을까 고민이 되네요 .. ❄...,4


In [ ]:
# data column 확인
data1.columns

Index(['문예진', '이상민', '정수연', '정승연', '황의린', '평균', '표준편차', '상품명', '리뷰', 'index'], dtype='object')

In [ ]:
# 불필요한 데이터 column 제거
df = data1[['평균', '리뷰',]]

In [ ]:
# column 순서 재배열
df = df[['리뷰', '평균']]

In [ ]:
# 데이터 확인
df.head()

,리뷰,평균
0,이브랜드는 올리브영 입점전부터 써본브랜드입니다\n 수분크림구매시에 성분이 중요한데요...,5.0
1,전에 어찌어찌하다 자작나무 수분크림 샘플을 받아 잘 쓰고 있는 중에\n 1+1행사한...,5.0
2,올리브영 대표 클린뷰티! 라운드랩!\n \n # 라운드랩 자작나무 수분 크림 \n ...,4.6
3,💚수분감\n 진짜 내가 찾던 수분만 있는 제품.. 유분은 기름종이 보시면 알겠지만 ...,3.8
4,원쁠원 행사 넘 좋아요 이번달 끝나기 전에 하나 더 사놓을까 고민이 되네요 .. ❄...,4.0


In [ ]:
# 평균 반올림
df['평균'] = df['평균'].round()

In [ ]:
df.rename(columns={'리뷰':'content', '평균':'label'}, inplace=True)

In [ ]:
df = df.astype({'label':'object'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  500 non-null    object
 1   label    500 non-null    object
dtypes: object(2)
memory usage: 7.9+ KB


In [ ]:
df['label'].value_counts()

3.0    154
4.0    139
2.0    139
1.0     62
5.0      6
Name: label, dtype: int64

In [ ]:
# label encoding
df.loc[(df['label'] == '1.0'), 'label'] = 0 
df.loc[(df['label'] == '2.0'), 'label'] = 1
df.loc[(df['label'] == '3.0'), 'label'] = 2 
df.loc[(df['label'] == '4.0'), 'label'] = 3 
df.loc[(df['label'] == '5.0'), 'label'] = 4 


# list형태로 데이터 저장 -> [content, label] 형태
data_list = []
for q, label in zip(df['content'], df['label'])  :
    data = []
    data.append(str(q)) # str타입으로 변환, 안 해주면 나중에 타입변환할 때 오류남
    data.append(str(label))

    data_list.append(data)

In [ ]:
df = df.astype({'label':'int'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  500 non-null    object
 1   label    500 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 7.9+ KB


In [ ]:
df['label_binary'] = [1 if h >= 3 else 0 for h in df['label']]

In [ ]:
df['label_binary'].value_counts()

1    299
0    201
Name: label_binary, dtype: int64

In [ ]:
# list형태로 데이터 저장 -> [content, label] 형태
data_list = []
for q, label in zip(df['content'], df['label_binary'])  :
    data = []
    data.append(str(q)) # str타입으로 변환, 안 해주면 나중에 타입변환할 때 오류남
    data.append(str(label))

    data_list.append(data)

In [ ]:
# 데이터 확인
print(data_list[0])
print(data_list[-1])

['이브랜드는 올리브영 입점전부터 써본브랜드입니다\n 수분크림구매시에 성분이 중요한데요\n 어떤수분크림은 향료나 발림성좋게하기위해 여드름일으키는 쉐어버터 등피이지 피피지 넣는데\n 요즘제품들은 소비자들이 워낙깐칸해서 이런 싸구려 쓰레기들 않넣고 중소기업든 제품들이 성분이 월등히 뛰어납니다\n 브랜드 광고보다 성분보세오ㅡ\n 수부지 ㆍ속건조ㆍ여드름피부추천입니다\n 남성분들도 쓰세요ㅡ추천\n 자작나무수액 몃프로 정확히 기재되어잇고\n 전체성분이 독하고 피부에 해가되는 성분이 없습니다\n 여름에 수분크림필수 쓰셔야 가을되면 피부건조잡티주름 안생깁니다\n 끈적임싫타고 안바르는분 이거바르세오ㅡ\n 가겹도 저렴하고 우리피부를 보호해주는 수분성분이 가든합니다\n 화정전쓰셔도ㅡ밀림없습니다\n 특히 악건성들은 이크림에 페이스오일 두방울 썩어 쓰시면됩니다\n \n 눈밑자글자글고민인분ㅡㅡㅡ꿀팁\n 이크림한번밀어바르고 페이스오일한방울석어서 또바르세오\n 그리고한번더두둘겨바르세요\n 머이렇게 많이 바르냐 하시겟죠\n 주름안생기려면 이렇게 관리하셔야합니ㅏ\n 저는이제품바르고 미백크림또바릅니다\n 그리고 아이크림또한번바릅니다\n 수분이잇어야미백효과가 작용하고 노화도멈춥니다\n 저는50이고10년전부터 올리브영신상들 다사서 성분만골라서 바릅니다\n 주름 안생김ㆍ잡티ㅡ선크림집에서도바름\n 기미없슴\n 자이제답나왓죠\n 수분크림이 제일중요합니다\n 추천', '1']
['처음에는 크림 롤링할때 알갱이가 느껴져서 이게 뭔가 싶었는데 계속 문대니 금새 사라지더라구요 지금같은 여름에 딱인 제품 같아요 확실히 이 크림 바르고 다음날 일어나보면 얼굴 붉은기도 진정된 느낌이구요 쓸수록 좋은 느낌을 받아요 토너랑 같이 짝궁템으로 여름에 쓰기 좋은거 같아요 독도 제품도 그렇구 순하고 자극 없어서 믿고 씁니다 🥰\n ※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 상품에 따라 용량 내지 일부 구성(1+1, 기획상품 등)이 상이할 수 있음을 안내드립니다.', '1']


In [ ]:
# train, test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
# 데이터 개수 확인
print(len(dataset_train))
print(len(dataset_test))

375
125


In [ ]:
# KoBERT모델의 입력으로 들어갈 수 있는 형태가 되도록 변환해주는 class
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        # BERTSentenceTransform이라는 모듈 사용
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        # 타입 변환
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    # item 가져오기
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    # 길이 출력
    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# 패딩
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model


In [ ]:
# 데이터 확인 -> array1은 시퀀스, array2는 길이와 타입, array3은 attention mask 시퀀스(1로 패딩된 값들은 연산할 필요가 없기 때문에, 연산을 하지 않아도 됨 -> 이걸 알려주는 데이터가 바로 얘)
data_train[0]

(array([   2,  517, 7267, 6756, 3298, 5920, 6117, 1849, 3111, 4158, 7098,
        2613, 7789, 3220, 6060, 7036, 5761,  517, 6010, 6029, 7096, 3531,
        3585, 7790, 4926, 3298, 5920, 6117, 6573, 4910, 6901, 4206, 5439,
        4924, 5585, 6023, 4955, 2307, 2856, 6797, 6369,  517, 7421, 7788,
        1119, 7867, 3673, 4209,  889, 3466, 3135, 2576, 3871, 5760, 7318,
        3461, 5176, 7953, 7874,    0, 3647, 4158, 7086,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [ ]:
# torch 형식의 dataset 생성
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# KoBERT 학습모델
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   # 클래스 수 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio) # 초기에 성능을 끌어올리는 것 = 운동할 때 워밍업 느낌

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0] # 맞은 비율
    return train_acc
    
train_dataloader

In [ ]:
# 모델 학습시키기
for e in range(30):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
     ->   out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.6339530348777771 train acc 0.625
epoch 1 train acc 0.6473484848484848


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 29 test acc 0.8008452868852459


  0%|          | 0/6 [00:00<?, ?it/s]

epoch 30 batch id 1 loss 0.0016500161727890372 train acc 1.0
epoch 30 train acc 1.0


  0%|          | 0/2 [00:00<?, ?it/s]

epoch 30 test acc 0.8004610655737705
